# 🎯 MLB Game Data Collection - Interactive Version

This notebook provides a **user-friendly, interactive way** to collect MLB game data:

1. **📅 Dynamic Date Selection** - Choose your date range interactively
2. **🔄 Two-Step Process** - Separate JSON download and CSV processing  
3. **⚡ Smart File Handling** - Automatically skips existing files
4. **📊 Enhanced Data** - Captures 60+ fields per game
5. **✅ Data Validation** - Comprehensive quality checks

## Quick Start:
1. Run the **Date Selection** cell to choose your date range
2. Run **Step 1** to download JSON data from MLB API 
3. Run **Step 2** to process JSON into enhanced CSV files
4. Review validation results and find your files in `data/csv/games/`

# Enhanced Game Range Collection Strategy

Based on the game-schedule.md documentation, the current implementation only captures a subset of available fields. Here's the strategy to capture all 60+ fields:

## Current Limitations
- Only captures 4 basic fields: `gamePk`, `gameDate`, `teams.away.team.id`, `teams.home.team.id`
- Filters only "Final" games, missing scheduled/in-progress games
- Doesn't capture team records, venue info, game status details, series info

## Enhanced Strategy
1. **Remove the "Final" filter** - capture all games regardless of status
2. **Use comprehensive field mapping** - extract all fields per game-schedule.md
3. **Add data validation** - ensure data quality and completeness
4. **Implement error handling** - robust API failure management
5. **Add progress tracking** - better monitoring of data collection

## 📅 Dynamic Date Selection

The cell below provides an interactive way to set your date range. You can either:
1. **Use default dates** - Press Enter to accept suggested dates
2. **Enter custom dates** - Type dates in YYYY-MM-DD format
3. **Choose preset ranges** - Select from common date ranges

In [2]:
# Define fetch_full_season_schedule if not already defined
def fetch_full_season_schedule(season=2024, save_path=None):
	"""
	Fetch the full MLB game schedule for a given season and save as JSON.
	Args:
		season (int): MLB season year (e.g., 2024)
		save_path (str): Optional path to save the JSON file
	Returns:
		dict: The schedule data
	"""
	import requests
	import json
	url = f"https://statsapi.mlb.com/api/v1/schedule?sportId=1&season={season}"
	print(f"Fetching MLB schedule for season {season}...")
	response = requests.get(url)
	response.raise_for_status()
	data = response.json()
	if save_path:
		with open(save_path, "w") as f:
			json.dump(data, f, indent=2)
		print(f"Saved schedule to {save_path}")
	return data

# Fetch and save the 2024 MLB game schedule as JSON
schedule_2024 = fetch_full_season_schedule(2024, "data/json/games/mlb_schedule_2024.json")
print(f"Total dates in 2024 schedule: {len(schedule_2024.get('dates', []))}")

Fetching MLB schedule for season 2024...
Saved schedule to data/json/games/mlb_schedule_2024.json
Total dates in 2024 schedule: 242
Saved schedule to data/json/games/mlb_schedule_2024.json
Total dates in 2024 schedule: 242


In [ ]:
# Determine the first and last date of the 2024 regular season
def get_regular_season_date_range(games):
    if not games:
        return None, None
    dates = [game.get('gameDate', '')[:10] for game in games if 'gameDate' in game]
    dates = sorted(dates)
    return dates[0], dates[-1] if dates else (None, None)

reg_start, reg_end = get_regular_season_date_range(regular_season_games)
print(f"2024 Regular Season Start Date: {reg_start}")
print(f"2024 Regular Season End Date:   {reg_end}")

In [3]:
# Filter for regular season games only (gameType = 'R')
regular_season_games = []
for date in schedule_2024.get('dates', []):
    for game in date.get('games', []):
        if game.get('gameType', '').upper() == 'R':
            regular_season_games.append(game)

print(f"Total regular season games in 2024: {len(regular_season_games)}")

Total regular season games in 2024: 2469


In [ ]:
import requests
import json

def fetch_full_season_schedule(season=2024, save_path=None):
    """
    Fetch the full MLB game schedule for a given season and save as JSON.
    Args:
        season (int): MLB season year (e.g., 2024)
        save_path (str): Optional path to save the JSON file
    Returns:
        dict: The schedule data
    """
    url = f"https://statsapi.mlb.com/api/v1/schedule?sportId=1&season={season}"
    print(f"Fetching MLB schedule for season {season}...")
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    if save_path:
        with open(save_path, "w") as f:
            json.dump(data, f, indent=2)
        print(f"Saved schedule to {save_path}")
    return data

# Example usage:
# schedule_2024 = fetch_full_season_schedule(2024, "data/json/games/mlb_schedule_2024.json")

## Fetch Full MLB Game Schedule for a Given Season
You can fetch the entire MLB game schedule for a specific season (e.g., 2024) using the MLB Stats API:
- Endpoint: `https://statsapi.mlb.com/api/v1/schedule?sportId=1&season=2024`
- This returns all games for the specified season in JSON format.

The code below demonstrates how to fetch and save the full season schedule for any year.

In [ ]:
import requests
import pandas as pd
import json
import os
from datetime import datetime, timedelta
import time
from pathlib import Path

# =============================================================================
# 📅 DYNAMIC DATE SELECTION - USER FRIENDLY INPUT
# =============================================================================

def get_date_input():
    """
    Interactive function to get start and end dates from user
    Returns tuple of (start_date, end_date) as datetime objects
    """
    
    print("🎯 MLB DATA COLLECTION - DATE RANGE SELECTION")
    print("=" * 60)
    
    # Default suggestions based on current date
    today = datetime.now()
    default_start = today - timedelta(days=7)  # Last week
    default_end = today - timedelta(days=1)    # Yesterday
    
    print(f"📅 Default suggestion (Press Enter to use):")
    print(f"   Game Schedule Start Date: {default_start.strftime('%Y-%m-%d')} (7 days ago)")
    print(f"   Game Schedule End Date:   {default_end.strftime('%Y-%m-%d')} (yesterday)")
    print()
    
    # Preset options
    presets = {
        '1': ('Yesterday only', today - timedelta(days=1), today - timedelta(days=1)),
        '2': ('Last 3 days', today - timedelta(days=3), today - timedelta(days=1)),
        '3': ('Last week', today - timedelta(days=7), today - timedelta(days=1)),
        '4': ('Current month', today.replace(day=1), today),
        '5': ('Last 30 days', today - timedelta(days=30), today - timedelta(days=1))
    }
    
    print("🎯 Quick preset options (automatically sets dates):")
    for key, (desc, start, end) in presets.items():
        print(f"   {key}. {desc}")
        print(f"      → Start: {start.strftime('%Y-%m-%d')}, End: {end.strftime('%Y-%m-%d')}")
    print()
    print("   6. Custom date range (you will enter dates manually)")
    print("   Enter = Use default suggestion above")
    print()
    
    # Get user choice
    while True:
        choice = input("Choose option (1-6) or press Enter for default: ").strip()
        
        if choice == "":
            # Use default
            start_date = default_start
            end_date = default_end
            print(f"\n✅ Using default range:")
            print(f"   Game Schedule Start Date: {start_date.strftime('%Y-%m-%d')}")
            print(f"   Game Schedule End Date:   {end_date.strftime('%Y-%m-%d')}")
            break
            
        elif choice in presets:
            # Use preset - automatically apply without asking for input
            desc, start_date, end_date = presets[choice]
            print(f"\n✅ Selected preset: {desc}")
            print(f"   Game Schedule Start Date: {start_date.strftime('%Y-%m-%d')}")
            print(f"   Game Schedule End Date:   {end_date.strftime('%Y-%m-%d')}")
            break
            
        elif choice == "6":
            # Custom input - only here do we ask for manual date entry
            print("\n📝 CUSTOM DATE ENTRY MODE")
            print("    You will now enter your own start and end dates")
            print("    Format required: YYYY-MM-DD (e.g., 2025-03-27)")
            print("=" * 50)
            
            # Get start date
            while True:
                try:
                    print("\n📅 GAME SCHEDULE START DATE")
                    print("   Examples of valid dates:")
                    print("   • 2025-03-27 (March 27, 2025)")
                    print("   • 2025-04-01 (April 1, 2025)") 
                    print("   • 2025-10-15 (October 15, 2025)")
                    start_input = input("\n   Enter Game Schedule Start Date (YYYY-MM-DD): ").strip()
                    
                    if not start_input:
                        print("❌ Please enter a start date - this field is required")
                        continue
                        
                    start_date = datetime.strptime(start_input, '%Y-%m-%d')
                    print(f"   ✅ Start Date Confirmed: {start_date.strftime('%A, %B %d, %Y')}")
                    break
                except ValueError:
                    print("❌ Invalid format. Please use YYYY-MM-DD format")
                    print("   Example: 2025-03-27 (not 03/27/2025 or 27-03-2025)")
                    print()
            
            # Get end date
            while True:
                try:
                    print("\n📅 GAME SCHEDULE END DATE")
                    print("   Examples of valid dates:")
                    print("   • 2025-04-30 (April 30, 2025)")
                    print("   • 2025-05-15 (May 15, 2025)")
                    print("   • 2025-10-31 (October 31, 2025)")
                    print(f"   Note: End date must be on or after {start_date.strftime('%Y-%m-%d')}")
                    end_input = input("\n   Enter Game Schedule End Date (YYYY-MM-DD): ").strip()
                    
                    if not end_input:
                        print("❌ Please enter an end date - this field is required")
                        continue
                        
                    end_date = datetime.strptime(end_input, '%Y-%m-%d')
                    
                    if end_date < start_date:
                        print("❌ End date must be on or after the start date")
                        print(f"   Your start date: {start_date.strftime('%Y-%m-%d')}")
                        print(f"   Your end date:   {end_date.strftime('%Y-%m-%d')}")
                        print()
                        continue
                        
                    print(f"   ✅ End Date Confirmed: {end_date.strftime('%A, %B %d, %Y')}")
                    break
                except ValueError:
                    print("❌ Invalid format. Please use YYYY-MM-DD format")
                    print("   Example: 2025-04-30 (not 04/30/2025 or 30-04-2025)")
                    print()
            
            print(f"\n✅ Custom MLB game schedule range confirmed:")
            print(f"   Game Schedule Start Date: {start_date.strftime('%Y-%m-%d')}")
            print(f"   Game Schedule End Date:   {end_date.strftime('%Y-%m-%d')}")
            break
            
        else:
            print(f"❌ Invalid choice '{choice}'. Please enter 1, 2, 3, 4, 5, 6, or press Enter")
            print()
    
    # Calculate and display summary
    days_count = (end_date - start_date).days + 1
    print(f"\n📊 MLB Game Schedule Summary:")
    print(f"   📅 Start: {start_date.strftime('%A, %B %d, %Y')}")
    print(f"   📅 End:   {end_date.strftime('%A, %B %d, %Y')}")
    print(f"   📈 Total days in range: {days_count}")
    print(f"   ⚾ Estimated MLB games: ~{days_count * 15} (avg 15 games/day)")
    print()
    
    return start_date, end_date

# Get the MLB game schedule date range from user
print("🎯 SETTING UP MLB GAME DATA COLLECTION")
print("=" * 60)
print("Please select the date range for MLB games you want to collect:")
print()

start_date, end_date = get_date_input()

🎯 SETTING UP MLB GAME DATA COLLECTION
Please select the date range for MLB games you want to collect:

🎯 MLB DATA COLLECTION - DATE RANGE SELECTION
📅 Default suggestion (Press Enter to use):
   Game Schedule Start Date: 2025-11-21 (7 days ago)
   Game Schedule End Date:   2025-11-27 (yesterday)

🎯 Quick preset options (automatically sets dates):
   1. Yesterday only
      → Start: 2025-11-27, End: 2025-11-27
   2. Last 3 days
      → Start: 2025-11-25, End: 2025-11-27
   3. Last week
      → Start: 2025-11-21, End: 2025-11-27
   4. Current month
      → Start: 2025-11-01, End: 2025-11-28
   5. Last 30 days
      → Start: 2025-10-29, End: 2025-11-27

   6. Custom date range (you will enter dates manually)
   Enter = Use default suggestion above



In [10]:
import requests
import pandas as pd
import json
import os
from datetime import datetime, timedelta
import time
from pathlib import Path

# =============================================================================
# PROCEDURE 1: JSON DATA EXTRACTION FROM API
# =============================================================================

def fetch_json_game_data(start_date, end_date, delay_seconds=1, overwrite=False):
    """
    Fetch raw JSON game data from MLB API and save to files.
    This procedure only hits the API and saves JSON files.
    
    Parameters:
    - start_date: datetime object for start date
    - end_date: datetime object for end date  
    - delay_seconds: delay between API calls (default 1 second)
    - overwrite: whether to overwrite existing files (default False)
    
    Returns:
    - success_dates: list of successfully processed dates
    - failed_dates: list of dates that failed
    """
    # Generate date range
    date_range = []
    current_date = start_date
    while current_date <= end_date:
        date_range.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=1)

    print(f"PROCEDURE 1: JSON DATA EXTRACTION")
    print(f"=" * 50)
    print(f"Fetching JSON data for {len(date_range)} dates: {date_range[0]} to {date_range[-1]}")
    print(f"API delay: {delay_seconds} seconds between requests")
    print(f"Overwrite existing: {overwrite}")
    
    # Create directories
    os.makedirs('data/json/games', exist_ok=True)
    
    success_dates = []
    failed_dates = []
    skipped_dates = []
    
    for i, date_str in enumerate(date_range):
        json_file = f'data/json/games/mlb_games_{date_str}.json'
        
        # Check if file exists and skip if not overwriting
        if not overwrite and os.path.exists(json_file):
            print(f"Skipping {date_str}... ({i+1}/{len(date_range)}) - File exists")
            skipped_dates.append(date_str)
            continue
            
        print(f"Fetching {date_str}... ({i+1}/{len(date_range)})")
        
        # MLB API endpoint with enhanced hydration
        url = f'https://statsapi.mlb.com/api/v1/schedule?sportId=1&date={date_str}&hydrate=game(content(summary,media(epg)),tickets),decisions'
        
        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()  # Raise exception for bad status codes
            data = response.json()
            
            # Save JSON file
            with open(json_file, 'w') as f:
                json.dump(data, f, indent=2)
            
            # Count games for feedback
            games_count = len(data.get('dates', [{}])[0].get('games', [])) if data.get('dates') else 0
            print(f"  ✓ Saved {games_count} games to {json_file}")
            success_dates.append(date_str)
                
        except requests.RequestException as e:
            print(f"  ✗ Network error for {date_str}: {e}")
            failed_dates.append(date_str)
        except Exception as e:
            print(f"  ✗ Error processing {date_str}: {e}")
            failed_dates.append(date_str)
        
        # Add delay to respect API rate limits
        if delay_seconds > 0 and i < len(date_range) - 1:
            time.sleep(delay_seconds)
    
    # Summary
    print(f"\n{'='*50}")
    print(f"JSON EXTRACTION SUMMARY")
    print(f"{'='*50}")
    print(f"Total dates requested: {len(date_range)}")
    print(f"Successfully fetched: {len(success_dates)}")
    print(f"Skipped (existing): {len(skipped_dates)}")
    print(f"Failed: {len(failed_dates)}")
    if failed_dates:
        print(f"Failed dates: {failed_dates}")
    print(f"JSON files location: data/json/games/")
    
    return success_dates, failed_dates, skipped_dates


# =============================================================================
# PROCEDURE 2: CSV PROCESSING FROM JSON FILES  
# =============================================================================

def enhance_game_data(games_list):
    """
    Enhanced function to extract all available fields from game data
    Based on game-schedule.md field specifications
    """
    enhanced_games = []
    
    for game in games_list:
        # Core game identification
        game_data = {
            'gamePk': game.get('gamePk'),
            'gameGuid': game.get('gameGuid'),
            'link': game.get('link'),
            'gameType': game.get('gameType'),
            'season': game.get('season'),
            'gameDate': game.get('gameDate'),
            'officialDate': game.get('officialDate'),
            'isTie': game.get('isTie'),
            'gameNumber': game.get('gameNumber'),
            'publicFacing': game.get('publicFacing'),
            'doubleHeader': game.get('doubleHeader'),
            'gamedayType': game.get('gamedayType'),
            'tiebreaker': game.get('tiebreaker'),
            'calendarEventID': game.get('calendarEventID'),
            'seasonDisplay': game.get('seasonDisplay'),
            'dayNight': game.get('dayNight'),
            'scheduledInnings': game.get('scheduledInnings'),
            'reverseHomeAwayStatus': game.get('reverseHomeAwayStatus'),
            'inningBreakLength': game.get('inningBreakLength'),
            'gamesInSeries': game.get('gamesInSeries'),
            'seriesGameNumber': game.get('seriesGameNumber'),
            'seriesDescription': game.get('seriesDescription'),
            'recordSource': game.get('recordSource'),
            'ifNecessary': game.get('ifNecessary'),
            'ifNecessaryDescription': game.get('ifNecessaryDescription'),
            
            # Game status fields
            'status.abstractGameState': game.get('status', {}).get('abstractGameState'),
            'status.codedGameState': game.get('status', {}).get('codedGameState'),
            'status.detailedState': game.get('status', {}).get('detailedState'),
            'status.statusCode': game.get('status', {}).get('statusCode'),
            'status.startTimeTBD': game.get('status', {}).get('startTimeTBD'),
            'status.abstractGameCode': game.get('status', {}).get('abstractGameCode'),
            
            # Away team information
            'teams.away.leagueRecord.wins': game.get('teams', {}).get('away', {}).get('leagueRecord', {}).get('wins'),
            'teams.away.leagueRecord.losses': game.get('teams', {}).get('away', {}).get('leagueRecord', {}).get('losses'),
            'teams.away.leagueRecord.pct': game.get('teams', {}).get('away', {}).get('leagueRecord', {}).get('pct'),
            'teams.away.score': game.get('teams', {}).get('away', {}).get('score'),
            'teams.away.team.id': game.get('teams', {}).get('away', {}).get('team', {}).get('id'),
            'teams.away.team.name': game.get('teams', {}).get('away', {}).get('team', {}).get('name'),
            'teams.away.team.link': game.get('teams', {}).get('away', {}).get('team', {}).get('link'),
            'teams.away.isWinner': game.get('teams', {}).get('away', {}).get('isWinner'),
            'teams.away.splitSquad': game.get('teams', {}).get('away', {}).get('splitSquad'),
            'teams.away.seriesNumber': game.get('teams', {}).get('away', {}).get('seriesNumber'),
            
            # Home team information
            'teams.home.leagueRecord.wins': game.get('teams', {}).get('home', {}).get('leagueRecord', {}).get('wins'),
            'teams.home.leagueRecord.losses': game.get('teams', {}).get('home', {}).get('leagueRecord', {}).get('losses'),
            'teams.home.leagueRecord.pct': game.get('teams', {}).get('home', {}).get('leagueRecord', {}).get('pct'),
            'teams.home.score': game.get('teams', {}).get('home', {}).get('score'),
            'teams.home.team.id': game.get('teams', {}).get('home', {}).get('team', {}).get('id'),
            'teams.home.team.name': game.get('teams', {}).get('home', {}).get('team', {}).get('name'),
            'teams.home.team.link': game.get('teams', {}).get('home', {}).get('team', {}).get('link'),
            'teams.home.isWinner': game.get('teams', {}).get('home', {}).get('isWinner'),
            'teams.home.splitSquad': game.get('teams', {}).get('home', {}).get('splitSquad'),
            'teams.home.seriesNumber': game.get('teams', {}).get('home', {}).get('seriesNumber'),
            
            # Venue information
            'venue.id': game.get('venue', {}).get('id'),
            'venue.name': game.get('venue', {}).get('name'),
            'venue.link': game.get('venue', {}).get('link'),
            
            # Content and other fields
            'content.link': game.get('content', {}).get('link'),
            'description': game.get('description'),
            'rescheduleDate': game.get('rescheduleDate'),
            'rescheduleGameDate': game.get('rescheduleGameDate'),
            'status.reason': game.get('status', {}).get('reason'),
            'rescheduledFrom': game.get('rescheduledFrom'),
            'rescheduledFromDate': game.get('rescheduledFromDate'),
            'resumeDate': game.get('resumeDate'),
            'resumeGameDate': game.get('resumeGameDate'),
            'resumedFrom': game.get('resumedFrom'),
            'resumedFromDate': game.get('resumedFromDate'),
        }
        
        enhanced_games.append(game_data)
    
    return enhanced_games

def process_json_to_csv(start_date, end_date, create_combined=True):
    """
    Process existing JSON files to create enhanced CSV files.
    This procedure only processes local JSON files - no API calls.
    
    Parameters:
    - start_date: datetime object for start date
    - end_date: datetime object for end date
    - create_combined: whether to create combined CSV file
    
    Returns:
    - processed_files: list of successfully processed files
    - missing_files: list of missing JSON files
    - combined_df: combined dataframe (if create_combined=True)
    """
    # Generate date range
    date_range = []
    current_date = start_date
    while current_date <= end_date:
        date_range.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=1)

    print(f"PROCEDURE 2: CSV PROCESSING FROM JSON")
    print(f"=" * 50)
    print(f"Processing JSON files for {len(date_range)} dates: {date_range[0]} to {date_range[-1]}")
    
    # Create directories
    os.makedirs('data/csv/games', exist_ok=True)
    
    all_games_data = []
    processed_files = []
    missing_files = []
    
    for i, date_str in enumerate(date_range):
        json_file = f'data/json/games/mlb_games_{date_str}.json'
        
        print(f"Processing {date_str}... ({i+1}/{len(date_range)})")
        
        if not os.path.exists(json_file):
            print(f"  ✗ JSON file not found: {json_file}")
            missing_files.append(date_str)
            continue
        
        try:
            # Load JSON file
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            # Extract games list
            games = data.get('dates', [{}])[0].get('games', []) if data.get('dates') else []
            
            if games:
                # Use enhanced data extraction
                enhanced_games = enhance_game_data(games)
                df = pd.DataFrame(enhanced_games)
                
                # Add processing metadata
                df['source_file'] = json_file
                df['processed_date'] = datetime.now().isoformat()
                
                # Save individual day CSV with all fields
                csv_file = f'data/csv/games/mlb_games_enhanced_{date_str}.csv'
                df.to_csv(csv_file, index=False)
                print(f"  ✓ Processed {len(df)} games → {csv_file}")
                
                # Add to combined dataset
                if create_combined:
                    all_games_data.append(df)
                    
                processed_files.append(date_str)
            else:
                print(f"  - No games found in {json_file}")
                processed_files.append(date_str)  # Still count as processed
                
        except Exception as e:
            print(f"  ✗ Error processing {json_file}: {e}")
            missing_files.append(date_str)
    
    # Create combined CSV
    combined_df = None
    if create_combined and all_games_data:
        combined_df = pd.concat(all_games_data, ignore_index=True)
        
        # Generate filename with date range
        start_str = start_date.strftime('%Y-%m-%d')
        end_str = end_date.strftime('%Y-%m-%d')
        combined_file = f'data/csv/games/mlb_games_enhanced_{start_str}_to_{end_str}.csv'
        
        combined_df.to_csv(combined_file, index=False)
        
        # Summary
        print(f"\n{'='*50}")
        print(f"CSV PROCESSING SUMMARY")
        print(f"{'='*50}")
        print(f"Total JSON files checked: {len(date_range)}")
        print(f"Successfully processed: {len(processed_files)}")
        print(f"Missing JSON files: {len(missing_files)}")
        if missing_files:
            print(f"Missing files for dates: {missing_files}")
        
        if combined_df is not None:
            print(f"Total games in combined file: {len(combined_df):,}")
            print(f"Total fields per game: {len(combined_df.columns)}")
            print(f"Combined file saved: {combined_file}")
            
            # Show game status distribution
            if 'status.detailedState' in combined_df.columns:
                print(f"\nGame Status Distribution:")
                status_counts = combined_df['status.detailedState'].value_counts()
                for status, count in status_counts.items():
                    print(f"  {status}: {count:,}")
    
    return processed_files, missing_files, combined_df

In [11]:
# =============================================================================
# 🔄 STEP 1: FETCH JSON DATA FROM MLB API (Using Dynamic Dates)
# =============================================================================

print("🔄 STEP 1: FETCHING JSON DATA FROM MLB API")
print("=" * 60)
print("⚠️  IMPORTANT: This will SKIP any existing JSON files")
print("   Set overwrite=True if you want to re-download existing files")
print("=" * 60)
print(f"📅 Using selected date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print()

# Fetch JSON data (automatically skips existing files when overwrite=False)
success_dates, failed_dates, skipped_dates = fetch_json_game_data(
    start_date=start_date, 
    end_date=end_date, 
    delay_seconds=1,        # 1 second delay between API calls
    overwrite=False         # 🚨 SET TO TRUE TO OVERWRITE EXISTING FILES
)

print(f"\n📊 Quick Summary:")
print(f"   ✅ New files downloaded: {len(success_dates)}")
print(f"   ⏭️  Files skipped (already exist): {len(skipped_dates)}")
print(f"   ❌ Failed downloads: {len(failed_dates)}")

if len(skipped_dates) > 0:
    print(f"\n📁 Skipped existing files for dates: {skipped_dates[:5]}{'...' if len(skipped_dates) > 5 else ''}")
    print(f"   💡 TIP: Set overwrite=True above to re-download these files")

🔄 STEP 1: FETCHING JSON DATA FROM MLB API
⚠️  IMPORTANT: This will SKIP any existing JSON files
   Set overwrite=True if you want to re-download existing files
📅 Using selected date range: 2025-03-27 to 2025-09-30

PROCEDURE 1: JSON DATA EXTRACTION
Fetching JSON data for 188 dates: 2025-03-27 to 2025-09-30
API delay: 1 seconds between requests
Overwrite existing: False
Skipping 2025-03-27... (1/188) - File exists
Skipping 2025-03-28... (2/188) - File exists
Skipping 2025-03-29... (3/188) - File exists
Skipping 2025-03-30... (4/188) - File exists
Skipping 2025-03-31... (5/188) - File exists
Skipping 2025-04-01... (6/188) - File exists
Skipping 2025-04-02... (7/188) - File exists
Skipping 2025-04-03... (8/188) - File exists
Skipping 2025-04-04... (9/188) - File exists
Skipping 2025-04-05... (10/188) - File exists
Skipping 2025-04-06... (11/188) - File exists
Skipping 2025-04-07... (12/188) - File exists
Skipping 2025-04-08... (13/188) - File exists
Skipping 2025-04-09... (14/188) - File 

In [12]:
# =============================================================================
# 🔄 STEP 2: PROCESS JSON FILES TO ENHANCED CSV (Using Dynamic Dates)
# =============================================================================

print("🔄 STEP 2: PROCESSING JSON FILES TO CSV")
print("=" * 60)
print(f"📅 Processing date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print("📊 This will create enhanced CSV files with 60+ fields per game")
print("=" * 60)
print()

# Process JSON files to enhanced CSV
processed_files, missing_files, enhanced_df = process_json_to_csv(
    start_date=start_date,
    end_date=end_date,
    create_combined=True
)

# Validate the data if we have results
if enhanced_df is not None and len(enhanced_df) > 0:
    print(f"\n{'='*60}")
    print("📊 DATA PROCESSING COMPLETE")
    print(f"{'='*60}")
    
    # Check if validation functions are available
    if 'validate_game_data' in dir() and 'generate_simplified_extract' in dir():
        print("Running data validation...")
        # Run validation
        missing_fields = validate_game_data(enhanced_df)
        
        # Create simplified extract  
        simplified_df = generate_simplified_extract(
            enhanced_df, 
            save_to_file=True, 
            start_date=start_date, 
            end_date=end_date
        )
    else:
        print("⚠️  Validation functions not loaded yet.")
        print("💡 Run cell 8 first to load validation functions, then re-run this cell for validation.")
    
    print(f"\n🎉 SUCCESS! Data collection and processing complete!")
    print(f"📁 Check the 'data/csv/games/' folder for your files")
    
else:
    print("\n⚠️  No data was processed. Make sure JSON files exist for the selected date range.")
    print("💡 Run STEP 1 first to download JSON data from the API.")

🔄 STEP 2: PROCESSING JSON FILES TO CSV
📅 Processing date range: 2025-03-27 to 2025-09-30
📊 This will create enhanced CSV files with 60+ fields per game

PROCEDURE 2: CSV PROCESSING FROM JSON
Processing JSON files for 188 dates: 2025-03-27 to 2025-09-30
Processing 2025-03-27... (1/188)
  ✓ Processed 14 games → data/csv/games/mlb_games_enhanced_2025-03-27.csv
Processing 2025-03-28... (2/188)
  ✓ Processed 9 games → data/csv/games/mlb_games_enhanced_2025-03-28.csv
Processing 2025-03-29... (3/188)
  ✓ Processed 15 games → data/csv/games/mlb_games_enhanced_2025-03-29.csv
Processing 2025-03-30... (4/188)
  ✓ Processed 13 games → data/csv/games/mlb_games_enhanced_2025-03-30.csv
Processing 2025-03-31... (5/188)
  ✓ Processed 14 games → data/csv/games/mlb_games_enhanced_2025-03-31.csv
Processing 2025-04-01... (6/188)
  ✓ Processed 13 games → data/csv/games/mlb_games_enhanced_2025-04-01.csv
Processing 2025-04-02... (7/188)
  ✓ Processed 15 games → data/csv/games/mlb_games_enhanced_2025-04-02.csv

## Data Quality Validation Functions

The following functions help validate the enhanced data collection:

In [ ]:
# =============================================================================
# ADDITIONAL UTILITY FUNCTIONS
# =============================================================================

def validate_game_data(df):
    """
    Comprehensive data validation for enhanced game dataset
    """
    print("ENHANCED DATA VALIDATION REPORT")
    print("=" * 50)
    
    # Basic statistics
    print(f"Total Records: {len(df):,}")
    print(f"Total Fields: {len(df.columns)}")
    print(f"Date Range: {df['gameDate'].min()} to {df['gameDate'].max()}")
    
    # Critical field validation
    critical_fields = [
        'gamePk', 'gameDate', 'teams.away.team.id', 'teams.home.team.id',
        'status.detailedState', 'venue.id'
    ]
    
    print(f"\nCritical Field Validation:")
    missing_critical = []
    for field in critical_fields:
        if field in df.columns:
            missing_count = df[field].isna().sum()
            if missing_count > 0:
                print(f"  ⚠️  {field}: {missing_count} missing values")
                missing_critical.append(field)
            else:
                print(f"  ✅ {field}: Complete")
        else:
            print(f"  ❌ {field}: Field not found!")
            missing_critical.append(field)
    
    # Game status analysis
    if 'status.detailedState' in df.columns:
        print(f"\nGame Status Distribution:")
        status_dist = df['status.detailedState'].value_counts()
        for status, count in status_dist.items():
            percentage = (count / len(df)) * 100
            print(f"  {status}: {count:,} ({percentage:.1f}%)")
    
    # Team data validation
    if 'teams.away.team.name' in df.columns and 'teams.home.team.name' in df.columns:
        away_teams = df['teams.away.team.name'].nunique()
        home_teams = df['teams.home.team.name'].nunique()
        print(f"\nTeam Coverage:")
        print(f"  Unique away teams: {away_teams}")
        print(f"  Unique home teams: {home_teams}")
    
    # Score data validation (only for final games)
    final_games = df[df['status.detailedState'] == 'Final'] if 'status.detailedState' in df.columns else df
    if len(final_games) > 0 and 'teams.away.score' in df.columns:
        score_missing = final_games[['teams.away.score', 'teams.home.score']].isna().any(axis=1).sum()
        print(f"\nScore Data (Final Games Only):")
        print(f"  Final games: {len(final_games):,}")
        print(f"  Missing scores: {score_missing}")
        if score_missing > 0:
            print(f"  ⚠️  Score completion: {((len(final_games) - score_missing) / len(final_games)) * 100:.1f}%")
    
    return missing_critical

def generate_simplified_extract(df, save_to_file=True, start_date=None, end_date=None):
    """
    Generate simplified extract based on game-schedule.md recommendations
    """
    print("\nGenerating Simplified Extract...")
    
    # Field mapping from complex to simplified
    field_mapping = {
        'gamePk': 'gamePk',
        'season': 'season', 
        'gameDate': 'gameDate',
        'officialDate': 'officialDate',
        'gameType': 'gameType',
        'status.abstractGameState': 'status',
        'teams.away.team.id': 'awayTeamId',
        'teams.away.team.name': 'awayTeamName',
        'teams.away.score': 'awayScore',
        'teams.away.leagueRecord.wins': 'awayWins',
        'teams.away.leagueRecord.losses': 'awayLosses',
        'teams.home.team.id': 'homeTeamId',
        'teams.home.team.name': 'homeTeamName', 
        'teams.home.score': 'homeScore',
        'teams.home.leagueRecord.wins': 'homeWins',
        'teams.home.leagueRecord.losses': 'homeLosses',
        'venue.id': 'venueId',
        'venue.name': 'venueName',
        'dayNight': 'dayNight',
        'seriesGameNumber': 'seriesGame',
        'gamesInSeries': 'totalGames',
        'doubleHeader': 'doubleHeader',
        'description': 'description'
    }
    
    # Create simplified dataframe
    simplified_df = pd.DataFrame()
    
    for complex_field, simple_field in field_mapping.items():
        if complex_field in df.columns:
            simplified_df[simple_field] = df[complex_field]
        else:
            simplified_df[simple_field] = None
            print(f"  ⚠️  Missing field: {complex_field}")
    
    # Add derived fields
    simplified_df['winner'] = df.apply(lambda row: 
        row['teams.home.team.name'] if row.get('teams.home.isWinner') 
        else row['teams.away.team.name'] if row.get('teams.away.isWinner')
        else None, axis=1
    )
    
    simplified_df['rescheduled'] = df['rescheduleDate'].notna() if 'rescheduleDate' in df.columns else False
    
    print(f"  ✅ Simplified extract created: {len(simplified_df)} records, {len(simplified_df.columns)} fields")
    
    # Save to file if requested
    if save_to_file and start_date and end_date:
        start_str = start_date.strftime('%Y-%m-%d')
        end_str = end_date.strftime('%Y-%m-%d')
        simplified_filename = f'data/csv/games/mlb_games_simplified_{start_str}_to_{end_str}.csv'
        simplified_df.to_csv(simplified_filename, index=False)
        print(f"  📁 Simplified file saved: {simplified_filename}")
    
    return simplified_df

def reprocess_existing_json_range(start_date, end_date):
    """
    Convenience function to reprocess existing JSON files in a date range
    without hitting the API again
    """
    print(f"REPROCESSING EXISTING JSON FILES")
    print(f"=" * 50)
    print(f"Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
    
    # Process JSON to CSV
    processed_files, missing_files, enhanced_df = process_json_to_csv(
        start_date=start_date,
        end_date=end_date,
        create_combined=True
    )
    
    if enhanced_df is not None:
        # Validate data
        missing_fields = validate_game_data(enhanced_df)
        
        # Generate simplified extract
        simplified_df = generate_simplified_extract(
            enhanced_df, 
            save_to_file=True, 
            start_date=start_date, 
            end_date=end_date
        )
        
        return enhanced_df, simplified_df, missing_fields
    else:
        return None, None, []

# ⚠️ REMOVED AUTOMATIC EXECUTION
# This section was causing duplicate file creation
print("📚 Validation and utility functions loaded successfully!")
print("💡 These functions are now available for use in other cells:")
print("   • validate_game_data(df)")
print("   • generate_simplified_extract(df, save_to_file, start_date, end_date)")
print("   • reprocess_existing_json_range(start_date, end_date)")
print()
print("✅ No files will be created automatically by loading this cell")


RUNNING DATA VALIDATION
ENHANCED DATA VALIDATION REPORT
Total Records: 378
Total Fields: 67
Date Range: 2025-09-01T17:05:00Z to 2025-10-01T01:08:00Z

Critical Field Validation:
  ✅ gamePk: Complete
  ✅ gameDate: Complete
  ✅ teams.away.team.id: Complete
  ✅ teams.home.team.id: Complete
  ✅ status.detailedState: Complete
  ✅ venue.id: Complete

Game Status Distribution:
  Final: 378 (100.0%)

Team Coverage:
  Unique away teams: 30
  Unique home teams: 30

Score Data (Final Games Only):
  Final games: 378
  Missing scores: 0

Generating Simplified Extract...
  ✅ Simplified extract created: 378 records, 25 fields
  📁 Simplified file saved: data/csv/games/mlb_games_simplified_2025-09-01_to_2025-09-30.csv


## Quick Reference Commands

Use these commands for common operations:

In [ ]:
# =============================================================================
# 🚀 QUICK REFERENCE COMMANDS (Using Dynamic Dates)
# =============================================================================

print("🚀 QUICK REFERENCE COMMANDS")
print("=" * 50)
print(f"Current date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print()
print("Uncomment any command below to run it:")
print()

# 1. RE-FETCH SPECIFIC DATES (overwrite existing JSON) 
# print("🔄 Re-downloading JSON files (overwriting existing)...")
# success_dates, failed_dates, skipped_dates = fetch_json_game_data(
#     start_date=start_date, 
#     end_date=end_date, 
#     overwrite=True
# )

# 2. REPROCESS EXISTING JSON FILES (no API calls)
# print("🔄 Reprocessing existing JSON files (no API calls)...")
# enhanced_df, simplified_df, missing_fields = reprocess_existing_json_range(
#     start_date=start_date, 
#     end_date=end_date
# )

# 3. PROCESS SINGLE DATE RANGE TO CSV ONLY
# print("🔄 Processing JSON to CSV only...")
# processed_files, missing_files, enhanced_df = process_json_to_csv(
#     start_date=start_date, 
#     end_date=end_date,
#     create_combined=True
# )

# 4. VALIDATE EXISTING DATAFRAME
# if 'enhanced_df' in locals() and enhanced_df is not None:
#     print("🔍 Validating existing data...")
#     missing_fields = validate_game_data(enhanced_df)
#     simplified_df = generate_simplified_extract(enhanced_df, True, start_date, end_date)

# 5. CHANGE DATE RANGE AND RESTART
# print("📅 To change dates, run the date selection cell again:")
# start_date, end_date = get_date_input()

print("✅ Quick reference commands loaded!")
print("💡 To change date range, re-run the date selection cell above")

Quick reference commands are ready to use!
Uncomment the commands you want to run.


In [9]:
import requests
import pandas as pd
import json
import os
from datetime import datetime, timedelta

# Define date range
start_date = datetime(2025, 9, 1)
end_date = datetime(2025, 9, 30)

# Generate list of dates
date_range = []
current_date = start_date
while current_date <= end_date:
    date_range.append(current_date.strftime('%Y-%m-%d'))
    current_date += timedelta(days=1)

print(f"Fetching data for dates: {date_range}")

all_games_data = []

for date_str in date_range:
    print(f"Processing {date_str}...")
    
    # MLB API endpoint for daily schedule
    url = f'https://statsapi.mlb.com/api/v1/schedule?sportId=1&date={date_str}'
    
    try:
        response = requests.get(url)
        data = response.json()
        
        # Create directories
        os.makedirs('data/json/games', exist_ok=True)
        os.makedirs('data/csv/games', exist_ok=True)
        
        # Save individual day JSON
        with open(f'data/json/games/mlb_games_{date_str}.json', 'w') as f:
            json.dump(data, f, indent=2)
        
        # Extract games list
        games = data.get('dates', [{}])[0].get('games', [])
        
        if games:
            df = pd.json_normalize(games)
            # Filter games where status.detailedState is 'Final'
            df = df[df['status.detailedState'] == 'Final']
            df = df[['gamePk', 'gameDate', 'teams.away.team.id', 'teams.home.team.id']]
            
            # Add date column for tracking
            df['date'] = date_str
            
            # Save individual day CSV
            df.to_csv(f'data/csv/games/mlb_games_{date_str}.csv', index=False)
            print(f"  - Saved {len(df)} games for {date_str}")
            
            # Add to combined dataset
            all_games_data.append(df)
        else:
            print(f"  - No games found for {date_str}")
            
    except Exception as e:
        print(f"  - Error fetching data for {date_str}: {e}")

# Combine all dates into one DataFrame
if all_games_data:
    combined_df = pd.concat(all_games_data, ignore_index=True)
    combined_df.to_csv('data/csv/games/mlb_games_2025-07-06_to_2025-07-18.csv', index=False)
    print(f"\nTotal games saved: {len(combined_df)}")
    print(f"Combined file saved: data/csv/games/mlb_games_2025-07-06_to_2025-07-18.csv")
else:
    print("No game data found for the date range")

Fetching data for dates: ['2025-09-01', '2025-09-02', '2025-09-03', '2025-09-04', '2025-09-05', '2025-09-06', '2025-09-07', '2025-09-08', '2025-09-09', '2025-09-10', '2025-09-11', '2025-09-12', '2025-09-13', '2025-09-14', '2025-09-15', '2025-09-16', '2025-09-17', '2025-09-18', '2025-09-19', '2025-09-20', '2025-09-21', '2025-09-22', '2025-09-23', '2025-09-24', '2025-09-25', '2025-09-26', '2025-09-27', '2025-09-28', '2025-09-29', '2025-09-30']
Processing 2025-09-01...
  - Saved 13 games for 2025-09-01
Processing 2025-09-02...
  - Saved 14 games for 2025-09-02
Processing 2025-09-03...
  - Saved 13 games for 2025-09-01
Processing 2025-09-02...
  - Saved 14 games for 2025-09-02
Processing 2025-09-03...
  - Saved 15 games for 2025-09-03
Processing 2025-09-04...
  - Saved 6 games for 2025-09-04
Processing 2025-09-05...
  - Saved 15 games for 2025-09-03
Processing 2025-09-04...
  - Saved 6 games for 2025-09-04
Processing 2025-09-05...
  - Saved 15 games for 2025-09-05
Processing 2025-09-06...
